<a href="https://colab.research.google.com/github/nivin18/RealTimeMLProjects/blob/master/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Libraries needed for NLP
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

#Libraries needed for Tensorflow processing
import tensorflow as tf
import numpy as np
import random
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
#load the intents.json file from your local device
from google.colab import files
files.upload()

Saving intents.json to intents.json


{'intents.json': b'{\r\n        "intents": [\r\n                {\r\n                        "tag": "greeting",\r\n                        "patterns": [\r\n                                "Hi",\r\n                                "How are you",\r\n                                "Is anyone there?",\r\n                                "Hello",\r\n                                "Good day"\r\n                        ],\r\n                        "responses": [\r\n                                "Hello, thanks for visiting",\r\n                                "Good to see you again",\r\n                                "Hi there, how can I help?"\r\n                        ],\r\n                        "context_set": ""\r\n                },\r\n                {\r\n                        "tag": "goodbye",\r\n                        "patterns": [\r\n                                "Bye",\r\n                                "See you later",\r\n                                "Goodbye"\r\n     

In [3]:
# import our chat-bot intents file
with open('intents.json') as json_data:
  intents = json.load(json_data)

In [4]:
intents


{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'chatbot',
   'patterns': ['Who built this chatbot?',
    'Tell me about Chatbot',
    'What is this chatbot name?'],
   'responses': ['Hi, I am Chatbot designed by Nivin.',
    'Thanks for asking. I am designed by Nivin A Thomas.',
    'I am a chatbot.']},
  {'tag': 'location',
   'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?'],
   'responses': ["We are from Wor

In [5]:
words = []
classes = []
documents = []
ignore = ['?']
# loop through each sentence in the intent's patterns
for intent in intents['intents']:
  for pattern in intent['patterns']:
    # tokenize each and every word in the sentence
    w = nltk.word_tokenize(pattern)
    # add word to the words list
    words.extend(w)
    # add word(s) to documents
    documents.append((w, intent['tag']))
    # add tags to our classes list
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

In [6]:
# Perform stemming and lower each word as well as remove duplicate
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

# remove duplicate classes
classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique stemmed words", words)

27 documents
8 classes ['about', 'chatbot', 'connect', 'goodbye', 'greeting', 'location', 'movies', 'thanks']
52 unique stemmed words ["'s", 'about', 'account', 'address', 'ani', 'anyon', 'are', 'built', 'bye', 'can', 'chatbot', 'connect', 'day', 'favourit', 'give', 'good', 'goodby', 'hello', 'help', 'hi', 'how', 'i', 'is', 'later', 'link', 'locat', 'me', 'media', 'movi', 'name', 'out', 'reach', 'recommend', 'see', 'social', 'some', 'suggest', 'tell', 'thank', 'that', 'there', 'thi', 'to', 'way', 'we', 'what', 'where', 'which', 'who', 'you', 'your', 'yourself']


In [7]:
# create training data
training = []
output = []
# create an empty array for output
output_empty = [0] * len(classes)

# create training set, bag of words for each sentence
for doc in documents:
  #initialize bag of words
  bag = []
  # list of tokenized words for the pattern
  pattern_words = doc[0]
  # stemming each words
  pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
  # create bag of words array
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)

    # output is '1' for current tag and '0' for the rest of other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

#shuffling features and turning it into np.array
random.shuffle(training)
training = np.array(training)

#creating training lists
train_x = list(training[:,0])
train_y = list(training[:,1])

<ipython-input-7-69abb7a2978d>:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [8]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10,input_shape=(len(train_x[0]),)))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Dense(len(train_y[0]), activation='softmax'))
model.compile(tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
model.fit(np.array(train_x), np.array(train_y), epochs=250, batch_size=8, verbose=1)
model.save("model.pk1")

Epoch 1/250
176/176 [==============================] - 1s 2ms/step - loss: 1.6962 - accuracy: 0.4281
Epoch 2/250
176/176 [==============================] - 0s 2ms/step - loss: 0.8111 - accuracy: 0.9530
Epoch 3/250
176/176 [==============================] - 0s 2ms/step - loss: 0.2520 - accuracy: 1.0000
Epoch 4/250
176/176 [==============================] - 0s 2ms/step - loss: 0.0811 - accuracy: 1.0000
Epoch 5/250
176/176 [==============================] - 0s 2ms/step - loss: 0.0356 - accuracy: 1.0000
Epoch 6/250
176/176 [==============================] - 0s 2ms/step - loss: 0.0195 - accuracy: 1.0000
Epoch 7/250
176/176 [==============================] - 0s 2ms/step - loss: 0.0122 - accuracy: 1.0000
Epoch 8/250
176/176 [==============================] - 0s 2ms/step - loss: 0.0083 - accuracy: 1.0000
Epoch 9/250
176/176 [==============================] - 0s 2ms/step - loss: 0.0059 - accuracy: 1.0000
Epoch 10/250
176/176 [==============================] - 0s 2ms/step - loss: 0.0044 - accura

In [10]:
import pickle
pickle.dump( {'words':words, 'classes':classes}, open( "training_data", "wb" ))

In [11]:
from keras.models import load_model
model = load_model("model.pk1")

In [12]:
# restoring all the data structures
data = pickle.load( open( "training_data", "rb"))
woords = data['words']
classes = data['classes']

In [13]:
with open('intents.json') as json_data:
  intents = json.load(json_data)

In [14]:
def clean_up_sentence(sentence):
  #tokenizing the pattern
  sentence_word = nltk.word_tokenize(sentence)
  #stemming each word
  sentence_word = [stemmer.stem(word.lower()) for word in sentence_word]
  return sentence_word

# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words):
  # tokenizing the pattern
  sentence_words = clean_up_sentence(sentence)
  # generating bag of words
  bag = [0]*len(words)
  for s in sentence_words:
    for i,w in enumerate(words):
      if w == s:
        bag[i] = 1
  bag = np.array(bag)
  return(bag)

In [15]:
from tensorflow.python.ops.gen_data_flow_ops import resource_accumulator_set_global_step
ERROR_THRESHOLD = 0.30
def classify(sentence):
  # generate probablities from the model
  bag = bow(sentence, words)
  results = model.predict(np.array([bag]))
  # filter out prediction below a threshold
  results = [[i,r] for i,r in enumerate(results[0]) if r> ERROR_THRESHOLD]
  # sort by strength of probablity
  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in results:
    return_list.append((classes[r[0]], r[1]))
  # return tuple of intent and probability
  return return_list

def response(sentence):
  results = classify(sentence)
  # if we have a classification then find the matching intent tag
  if results:
    # loop as long as there are matches to process
    while resource_accumulator_set_global_step:
      for i in intents['intents']:
        # find a tag matching the first result
        if i['tag'] == results[0][0]:
          # a random response from the intent
          return print(random.choice(i['responses']))

      results.pop(0)


In [16]:
response('Where are you located?')

1/1 [==============================] - 0s 88ms/step
Thans for your Interest. I live in India.


In [17]:
response('hi how are you?')

1/1 [==============================] - 0s 26ms/step
Hi there, how can I help?
